In [110]:
import functools
import operator
import itertools
import math
import sys
import json
import re
from enum import Enum

data = open("input.txt").read().splitlines()
coords = [tuple([int(x) for x in line.split(',')]) for line in data]
directions = set([
    (1,0,0),
    (0,1,0),
    (0,0,1),
    (-1,0,0),
    (0,-1,0),
    (0,0,-1)
    ]) 
dims = [0,1,2]
def add(a,b):
    return tuple([x + y for x,y in zip(a,b)])

class Kind(Enum):
    UNKNOWN = 0
    SOLID = 1
    INTERIOR = 2
    EXTERIOR = 3

class Droplet():
    def __init__(self, coords) -> None:
        self.min = (math.inf,math.inf,math.inf)
        self.max = (0,0,0)
        self.occupied = coords
        self.voxels = {}
        self.surface_area = 0
        for c in coords:
            self.include(c)
        self._compute_surface_area()
        self.debug = False

    def include(self, coord):
        self.min = tuple([min(self.min[x],coord[x]) for x in dims])
        self.max = tuple([max(self.max[x],coord[x]) for x in dims])
        self.voxels[coord] = Kind.SOLID

    def in_box(self, coord):
        return all([coord[x]>=self.min[x] for x in dims]) and \
               all([coord[x]<=self.max[x] for x in dims])

    def _compute_surface_area(self):
        for cell in self.occupied:
            for direction in directions:
                adjacent = add(cell, direction)
                kind = self.get_kind(adjacent)
                if kind == Kind.EXTERIOR:
                    self.surface_area += 1


    def get_kind(self, coord, path = []):
        kind = self.voxels.get(coord, Kind.UNKNOWN) 
        #print(f"{'  '*len(path)}get_kind({coord},{path})")
        if kind == Kind.UNKNOWN:
            if not self.in_box(coord):
                kind = Kind.EXTERIOR
            else:
                for d in directions:
                    adjacent = add(coord, d)
                    if adjacent in path:
                        continue
                    if self.get_kind(adjacent, path + [coord]) == Kind.EXTERIOR:
                        kind = Kind.EXTERIOR
                        break
                if kind == Kind.UNKNOWN:
                    kind = Kind.INTERIOR
        
            if len(path) == 0 or kind == Kind.EXTERIOR:
                self.voxels[coord] = kind
        
        #print(f"{' '*len(path)} -> {kind}")
        return kind

box = set(
      [(x,0,0) for x in range(0,5)] + 
      [(x,1,0) for x in range(0,5)] + 
      [(x,2,0) for x in range(0,5)] + 
      [(x,0,1) for x in range(0,5)] + 
      [(x,1,1) for x in range(0,5)] + 
      [(x,2,1) for x in range(0,5)] + 
      [(x,0,2) for x in range(0,5)] + 
      [(x,1,2) for x in range(0,5)] + 
      [(x,2,2) for x in range(0,5)]
   )

box.remove((0,1,1))
box.remove((1,1,1))
box.remove((2,1,1))
box.add((-1,1,0))
box.add((-2,1,0))
box.add((-2,1,1))

droplet = Droplet(coords)
print(droplet.get_kind((1,1,1)))

print(droplet.surface_area)

KeyboardInterrupt: 